# Back to SQL

We're back to working with plain ol' SQL.

In [28]:
import sqlite3

In [29]:
db = sqlite3.connect('library.db')

**All our current books**
These are our current books in the DB:

In [4]:
cursor = db.execute('SELECT * FROM Book')
cursor.fetchall()

[(1, 3, 'Pride & Prejudice', 'B1'),
 (2, 4, 'A Study in Scarlet', 'B2'),
 (3, 3, 'Emma', 'B3'),
 (8, 1, 'The Raven', 'B4'),
 (9, 5, 'Ficciones', 'B5')]

##### Requirement: Add a popularity field to Books

We need now to add a new **required** field to the book table: `popularity`. How can we do it? SQL has a simple `ALTER TABLE` command that let's you add the column:

In [5]:
db.execute('ALTER TABLE Book ADD COLUMN popularity text not null')

OperationalError: Cannot add a NOT NULL column with default value NULL

But as the column is required (`NOT NULL`), the command fails. We either need to specify a default value, or make it `NULL`able (which **is not** our requirement). Let's start with the first option, for simplicity:

In [6]:
db.execute('ALTER TABLE Book ADD COLUMN popularity text not null default 1.99')

That seems to work... Let's explore all our books:

In [7]:
cursor = db.execute('SELECT * FROM Book')
cursor.fetchall()

[(1, 3, 'Pride & Prejudice', 'B1', '1.99'),
 (2, 4, 'A Study in Scarlet', 'B2', '1.99'),
 (3, 3, 'Emma', 'B3', '1.99'),
 (8, 1, 'The Raven', 'B4', '1.99'),
 (9, 5, 'Ficciones', 'B5', '1.99')]

### A more challenging requirement

Our requirement now has changed. We can't provide a fixed default value for the new column (`1.99` in our previous example). The value to backfill the current books should be based according to the length of the title. The longer the title, the higher the popularity of the book.  `¯\_(ツ)_/¯`.

**Important:** Make sure you reset the database before trying the following example.

**Solution**

In this case we could create the new column as `NULL`able, insert the popularities, and then make it `NOT NULL`. The process would be as follows:

In [31]:
db = sqlite3.connect('library.db')

**Step 1: ALTER TABLE nullable**

In [30]:
db.execute('ALTER TABLE Book ADD COLUMN popularity text')

**Step 2: Compute popularity for each book**

In [25]:
def popularity(books):
    title_lengths = [len(title) for id, title in books]
    max_length = max(title_lengths)
    popularities = []
    for id, title in books:
        popularity = round(len(title) / max_length * 5, 2)
        popularities.append((id, title, popularity))
    return popularities

In [15]:
cursor = db.execute('SELECT id, title FROM Book')
cursor.fetchall()

[(1, 'Pride & Prejudice'),
 (2, 'A Study in Scarlet'),
 (3, 'Emma'),
 (8, 'The Raven'),
 (9, 'Ficciones')]

In [32]:
cursor = db.execute('SELECT id, title FROM Book')
popularity = popularity(cursor.fetchall())
popularity

[(1, 'Pride & Prejudice', 4.72),
 (2, 'A Study in Scarlet', 5.0),
 (3, 'Emma', 1.11),
 (8, 'The Raven', 2.5),
 (9, 'Ficciones', 2.5)]

**Step 3: Update each book with its corresponding popularity**

In [34]:
for id, name, pop in popularity:
    db.execute('UPDATE Book SET (popularity) = (:pop) WHERE id = :id', {
        'id': id,
        'pop': pop
    })


Check popularities are correctly set:

In [35]:
cursor = db.execute('SELECT id, title, popularity FROM Book')
cursor.fetchall()

[(1, 'Pride & Prejudice', '4.72'),
 (2, 'A Study in Scarlet', '5.0'),
 (3, 'Emma', '1.11'),
 (8, 'The Raven', '2.5'),
 (9, 'Ficciones', '2.5')]

**Step 4: Make the column `NOT NULL`**

Homework. This is too complicated 😅.

Let's better use Django's ORM...